In [1]:
import os
import requests

name = "opj_v04_test.py"
if not os.path.exists(name):
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/inlp_repo/main/labs/opj_v04_model_jezika/{name}")
    with open(name, "wb") as fp:
        fp.write(response.content)
    response.close()
    
name = "alan_ford_001_grupa_tnt.txt"
if not os.path.exists(name):
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/inlp_repo/main/labs/opj_v04_model_jezika/{name}")
    with open(name, "wb") as fp:
        fp.write(response.content)
    response.close()

In [2]:
import re
from opj_v04_test import *

def read_file(filename):
    return open(filename, "r", encoding="utf8").read()

# 4. Model jezika

## 4.1 segmentiranje rečenice

Napravi funkciju **segment_sent()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **txt** tekst
* izlaz:
    * lista rečenica dobivenih iz teksta razbijanjem po regularnom izrazu koji traži sve **interpunkcijske znakove** i **nove redove**. Također se izbacuju sve vrste **zagrada** i **zareza** na početku i/ili kraju svake rečenice.

In [3]:
def segment_sent(txt):
    sentences = re.split(r'[\n.\!\?]', txt.strip())
    sentences = [sent.strip(' [](){},') for sent in sentences if sent.strip()]
    return sentences
    
testname(segment_sent)
test(segment_sent, ("A gdje je tvoj ured?", ), ['A gdje je tvoj ured'])
test(segment_sent, ("A gdje je tvoj ured? Ne znam gdje je.", ), ['A gdje je tvoj ured', 'Ne znam gdje je'])
test(segment_sent, ("""[Alan Ford 001 - Grupa TNT]
New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna mrava što stanuju...
Čujmo malo muzike! Tišina me odviše podsjeća na groblje! """, ),
['Alan Ford 001 - Grupa TNT',
'New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna mrava što stanuju',
'Čujmo malo muzike',
'Tišina me odviše podsjeća na groblje'])


SEGMENT_SENT
------------------------------------------------------------
OK	segment_sent('A gdje je tvoj ured?',) 
=> ['A gdje je tvoj ured']
== ['A gdje je tvoj ured']

OK	segment_sent('A gdje je tvoj ured? Ne znam gdje je.',) 
=> ['A gdje je tvoj ured', 'Ne znam gdje je']
== ['A gdje je tvoj ured', 'Ne znam gdje je']

OK	segment_sent('[Alan Ford 001 - Grupa TNT]\nNew York, najveći grad sjedinjenih država, kip slobo...',) 
=> ['Alan Ford 001 - Grupa TNT', 'New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna...', 'Čujmo malo muzike', 'Tišina me odviše podsjeća na groblje']
== ['Alan Ford 001 - Grupa TNT', 'New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna mrava što stanuju', 'Čujmo malo muzike', 'Tišina me odviše podsjeća na groblje']



## 4.2 Segmentiranje riječi

Napravi funkciju **segment_word()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **txt** tekst
* izlaz:
    * lista riječi u tekstu nastalih razbijanjem po regularnom izrazu koji traži sve **razmake**, **interpunkcijske znakove** i **zareze**. Također se izbacuju sve vrste **zagrada**, **navodika** i **dvotočki** na početku i/ili kraju svake riječi.

In [4]:
def segment_word(txt):
    words = re.split(r'[ \,\.\!\?]', txt)
    words = [word.strip('[](){}"\':') for word in words if word.strip()]
    return words

testname(segment_word)
test(segment_word, ("A gdje je tvoj ured?", ), ['A', 'gdje', 'je', 'tvoj', 'ured'])
test(segment_word, ("A gdje je tvoj ured? Ne znam gdje je.", ), ['A', 'gdje', 'je', 'tvoj', 'ured', 'Ne', 'znam', 'gdje', 'je'])
test(segment_word, ("""[Alan Ford 001 - Grupa TNT]
New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna mrava što stanuju...
Čujmo malo muzike! Tišina me odviše podsjeća na groblje! """, ),
['Alan', 'Ford', '001', '-', 'Grupa', 'TNT', 'New', 'York', 'najveći', 'grad', 'sjedinjenih', 'država', 'kip', 'slobode', 'neboderi', 'devet', 'milijuna', 'mrava', 'što', 'stanuju', 'Čujmo', 'malo', 'muzike', 'Tišina', 'me', 'odviše', 'podsjeća', 'na', 'groblje'])



SEGMENT_WORD
------------------------------------------------------------
OK	segment_word('A gdje je tvoj ured?',) 
=> ['A', 'gdje', 'je', 'tvoj', 'ured']
== ['A', 'gdje', 'je', 'tvoj', 'ured']

OK	segment_word('A gdje je tvoj ured? Ne znam gdje je.',) 
=> ['A', 'gdje', 'je', 'tvoj', 'ured', 'Ne', 'znam', 'gdje', 'je']
== ['A', 'gdje', 'je', 'tvoj', 'ured', 'Ne', 'znam', 'gdje', 'je']

X	segment_word('[Alan Ford 001 - Grupa TNT]\nNew York, najveći grad sjedinjenih država, kip slobo...',) 
=> ['Alan', 'Ford', '001', '-', 'Grupa', 'TNT', 'New', 'York', 'najveći', 'grad', 'sjedinjenih', 'država', 'kip', 'slobode', 'neboderi', 'devet', 'milijuna', 'mrava', 'što', 'stanuju', 'Čujmo', 'malo', 'muzike', 'Tišina', 'me', 'odviše', 'podsjeća', 'na', 'groblje']
!= ['Alan', 'Ford', '001', '-', 'Grupa', 'TNT]\nNew', 'York', 'najveći', 'grad', 'sjedinjenih', 'država', 'kip', 'slobode', 'neboderi', 'devet', 'milijuna', 'mrava', 'što', 'stanuju', '\nČujmo', 'malo', 'muzike', 'Tišina', 'me', 'odviše',

## 4.3. Izgradnja rječnika

Napravi funkciju **build_vocab()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **txt** tekst
* izlaz:
    * skup riječi iz teksta. Koristiti funkcije **segment_sent()** i **segment_word()** za određivanje rečenica i riječi u tekstu.

In [5]:
def build_vocab(txt):
    vocab = set()
    for sent in segment_sent(txt):
        for word in segment_word(sent):
            vocab.add(word)
    return vocab

testname(build_vocab)
test(build_vocab, ("A gdje je tvoj ured?", ), {'A', 'gdje', 'je', 'tvoj', 'ured'})
test(build_vocab, ("A gdje je tvoj ured? Ne znam gdje je.", ), {'znam', 'A', 'Ne', 'gdje', 'je', 'tvoj', 'ured'})
test(build_vocab, ("""[Alan Ford 001 - Grupa TNT]
New York, najveći grad sjedinjenih država, kip slobode, neboderi, devet milijuna mrava što stanuju...
Čujmo malo muzike! Tišina me odviše podsjeća na groblje! """, ),
{'država', 'muzike', 'Alan', 'malo', 'York', 'odviše', 'Grupa', 'TNT', 'najveći', 'groblje', 'grad', '001', 'New', 'što', 'me', 'milijuna', 'devet', 'mrava', 'sjedinjenih', '-', 'Čujmo', 'Ford', 'Tišina', 'kip', 'neboderi', 'stanuju', 'na', 'podsjeća', 'slobode'})
            


BUILD_VOCAB
------------------------------------------------------------
OK	build_vocab('A gdje je tvoj ured?',) 
=> {'ured', 'je', 'A', 'tvoj', 'gdje'}
== {'ured', 'je', 'A', 'tvoj', 'gdje'}

OK	build_vocab('A gdje je tvoj ured? Ne znam gdje je.',) 
=> {'ured', 'je', 'A', 'znam', 'tvoj', 'gdje', 'Ne'}
== {'ured', 'je', 'A', 'znam', 'tvoj', 'gdje', 'Ne'}

OK	build_vocab('[Alan Ford 001 - Grupa TNT]\nNew York, najveći grad sjedinjenih država, kip slobo...',) 
=> {'-', 'slobode', 'me', 'što', 'najveći', 'kip', 'Ford', 'muzike', 'New', 'podsjeća', 'grad', '001', 'milijuna', 'stanuju', 'odviše', 'devet', 'mrava', 'York', 'neboderi', 'groblje', 'malo', 'sjedinjenih', 'Čujmo', 'Tišina', 'TNT', 'Grupa', 'država', 'na', 'Alan'}
== {'-', 'slobode', 'me', 'što', 'najveći', 'kip', 'Ford', 'muzike', 'New', 'podsjeća', 'grad', '001', 'milijuna', 'stanuju', 'odviše', 'devet', 'mrava', 'York', 'neboderi', 'groblje', 'malo', 'sjedinjenih', 'Čujmo', 'Tišina', 'TNT', 'Grupa', 'država', 'na', 'Alan'}



## 4.4 Izgradnja n-grama

Napravi funkciju **build_sent_ngram()** po sljedećim ulaznim i izlaznim podacima
* ulaz:
    * **sent** tekst rečenice
    * **ngram_size** broj koji određuje veličinu n-grama
* izlaz:
    * lista n-grama zadane rečenice. n-gram je uređena n-torka (tuple) riječi. 
    
Bitni su početak i kraj rečenice. Početak i kraj rečenice označiti sa `<sent>` i `</sent>`. 

**Napomena**: kod unigrama odnosno kad je ngram_size = 1 n-torka je i dalje tuple, odnosno ima oblik (riječ, ).

In [6]:
def build_sent_ngram(sent, ngram_size):
    ngram = []
    sent_words = ['<sent>'] + segment_word(sent) + ['</sent>']
    for i in range(len(sent_words) - ngram_size + int(ngram_size > 0)):
        ngram.append(tuple(sent_words[i:i + ngram_size]))
    return ngram

testname(build_sent_ngram)
test(build_sent_ngram, ("A gdje je tvoj ured?", 1), [('<sent>',), ('A',), ('gdje',), ('je',), ('tvoj',), ('ured',), ('</sent>',)])
test(build_sent_ngram, ("A gdje je tvoj ured?", 2), [('<sent>', 'A'), ('A', 'gdje'), ('gdje', 'je'), ('je', 'tvoj'), ('tvoj', 'ured'), ('ured', '</sent>')])
test(build_sent_ngram, ("A gdje je tvoj ured?", 5), [('<sent>', 'A', 'gdje', 'je', 'tvoj'), ('A', 'gdje', 'je', 'tvoj', 'ured'), ('gdje', 'je', 'tvoj', 'ured', '</sent>')])


BUILD_SENT_NGRAM
------------------------------------------------------------
OK	build_sent_ngram('A gdje je tvoj ured?', 1) 
=> [('<sent>',), ('A',), ('gdje',), ('je',), ('tvoj',), ('ured',), ('</sent>',)]
== [('<sent>',), ('A',), ('gdje',), ('je',), ('tvoj',), ('ured',), ('</sent>',)]

OK	build_sent_ngram('A gdje je tvoj ured?', 2) 
=> [('<sent>', 'A'), ('A', 'gdje'), ('gdje', 'je'), ('je', 'tvoj'), ('tvoj', 'ured'), ('ured', '</sent>')]
== [('<sent>', 'A'), ('A', 'gdje'), ('gdje', 'je'), ('je', 'tvoj'), ('tvoj', 'ured'), ('ured', '</sent>')]

OK	build_sent_ngram('A gdje je tvoj ured?', 5) 
=> [('<sent>', 'A', 'gdje', 'je', 'tvoj'), ('A', 'gdje', 'je', 'tvoj', 'ured'), ('gdje', 'je', 'tvoj', 'ured', '</sent>')]
== [('<sent>', 'A', 'gdje', 'je', 'tvoj'), ('A', 'gdje', 'je', 'tvoj', 'ured'), ('gdje', 'je', 'tvoj', 'ured', '</sent>')]



## 4.5 Prebrojavanje n-grama

Napravi funkciju **count_ngrams()** po sljedećim ulaznim i izlaznim podacima
* ulaz
    * **txt** tekst
    * **ngram_size** broj koji određuje veličinu n-grama    
* izlaz
    * rječnik čiji su ključevi ngrami (kao tuple riječi), a vrijednosti broj pojavljivanja (frekvencija) u tekstu
    
Napomena: bitni su početak i kraj rečenice. Početak i kraj rečenice označiti sa `<sent>` i `</sent>`. Koristiti prethodno definiranu funkciju **build_sent_ngram()**.

Npr. Za tekst "A gdje je ured? Ne znam gdje je." i za ngram_size = 2 dobiveni ngrami su

    ("<sent>", "A"), ("A", "gdje"), ("gdje", "je"), ("je", "ured"), ("ured", "</sent>")
    ("<sent>", "Ne"), ("Ne", "znam"), ("znam", "gdje"), ("gdje", "je"), ("je", "</sent>")

a, brojač je

    {("<sent>", "A"): 1,
     ("A", "gdje"): 1,
     ("gdje", "je"): 2,
     ("je", "ured"): 1,
     ("ured", "</sent>"): 1,
     ("<sent>", "Ne"): 1,
     ("Ne", "znam"): 1,
     ("znam", "gdje"): 1,
     ("je", "</sent>": 1)
     }

In [7]:
def count_ngrams(txt, ngram_size):
    counter = {}
    for sent in segment_sent(txt):
        for ngram in build_sent_ngram(sent, ngram_size):
            counter[ngram] = counter.get(ngram, 0) + 1
    return counter

FILENAME = 'alan_ford_001_grupa_tnt.txt'
testname(count_ngrams)
test(count_ngrams, ("A gdje je tvoj ured? Ne znam gdje je.", 1), {('<sent>',): 2, ('A',): 1, ('gdje',): 2, ('je',): 2, ('tvoj',): 1, ('ured',): 1, ('</sent>',): 2, ('Ne',): 1, ('znam',): 1})
test(count_ngrams, ("A gdje je tvoj ured? Ne znam gdje je.", 2), {('<sent>', 'A'): 1, ('A', 'gdje'): 1, ('gdje', 'je'): 2, ('je', 'tvoj'): 1, ('tvoj', 'ured'): 1, ('ured', '</sent>'): 1, ('<sent>', 'Ne'): 1, ('Ne', 'znam'): 1, ('znam', 'gdje'): 1, ('je', '</sent>'): 1})
counter1 = test_model(count_ngrams, (read_file(FILENAME), 1), {1: 1180, 2: 183, 3: 73, 4: 44, 5: 20, 6: 8, 7: 15, 8: 13, 9: 4, 10: 3, 11: 2, 12: 3, 13: 1, 14: 3, 15: 4, 16: 2, 17: 1, 18: 1, 19: 4, 20: 1, 22: 1, 27: 1, 31: 1, 34: 1, 35: 1, 40: 1, 46: 1, 47: 1, 55: 1, 97: 1, 101: 1, 109: 1, 688: 2})
counter2 = test_model(count_ngrams, (read_file(FILENAME), 2), {1: 2910, 2: 220, 3: 61, 4: 30, 5: 10, 6: 6, 7: 9, 8: 6, 9: 3, 10: 4, 14: 2, 15: 3, 20: 1, 22: 1})
counter3 = test_model(count_ngrams, (read_file(FILENAME), 3), {1: 3055, 2: 78, 3: 19, 4: 7, 6: 3, 7: 2, 8: 2})


COUNT_NGRAMS
------------------------------------------------------------
OK	count_ngrams('A gdje je tvoj ured? Ne znam gdje je.', 1) 
=> {('<sent>',): 2, ('A',): 1, ('gdje',): 2}
== {('<sent>',): 2, ('A',): 1, ('gdje',): 2, ('je',): 2, ('tvoj',): 1, ('ured',): 1, ('</sent>',): 2, ('Ne',): 1, ('znam',): 1}

OK	count_ngrams('A gdje je tvoj ured? Ne znam gdje je.', 2) 
=> {('<sent>', 'A'): 1, ('A', 'gdje'): 1, ('gdje', 'je'): 2}
== {('<sent>', 'A'): 1, ('A', 'gdje'): 1, ('gdje', 'je'): 2, ('je', 'tvoj'): 1, ('tvoj', 'ured'): 1, ('ured', '</sent>'): 1, ('<sent>', 'Ne'): 1, ('Ne', 'znam'): 1, ('znam', 'gdje'): 1, ('je', '</sent>'): 1}

OK	count_ngrams('[Alan Ford 001 - Grupa TNT]\n\nNew York, najveći grad sjedinjenih država, kip slob...', 1) 
=> {1: 1180, 2: 183, 3: 73, 4: 44, 5: 20, 6: 8, 7: 15, 8: 13, 9: 4, 10: 3, 11: 2, 12: 3, 13: 1, 14: 3, 15: 4, 16: 2, 17: 1, 18: 1, 19: 4, 20: 1, 22: 1, 27: 1, 31: 1, 34: 1, 35: 1, 40: 1, 46: 1, 47: 1, 55: 1, 97: 1, 101: 1, 109: 1, 688: 2}
== {1: 1180

## 4.5. Izgradnja modela jezika

Napravi funkciju **build_lang_model()** po sljedećim ulaznim i izlaznim podacima
* ulaz
    * **txt** tekst
    * **ngram_size** broj koji određuje veličinu n-grama
* izlaz
    * rječnik čiji su ključevi ngrami (kao tuple riječi), a vrijednosti broj pojavljivanja (frekvencija) u tekstu
    
    
Napomena: za bigrame vjerojatnost pojedinog bigrama (A, B) je jednaka broju pojavljivanja bigrama (A, B) podijeljenog s brojem pojavljivanja unigrama (A, )

$$P(A, B) = \frac{broj((A, B))}{broj(A)}$$    
    
Napomena: za trigrame vjerojatnost pojedinih trigrama (A, B, C) je jednaka broju pojavljivanja trigrama (A, B, C) podijeljenog s brojem pojavlivanja bigrama (A, B)

$$P(A, B, C) = \frac{broj((A, B, C))}{broj(A, B)}$$    

Npr. Za tekst "A gdje je ured? Ne znam gdje je." i za ngram_size = 2 dobiveni bigrami su

    ("<sent>", "A"), ("A", "gdje"), ("gdje", "je"), ("je", "ured"), ("ured", "</sent>")
    ("<sent>", "Ne"), ("Ne", "znam"), ("znam", "gdje"), ("gdje", "je"), ("je", "</sent>")

brojač bigrama je 

    {("<sent>", "A"): 1,
     ("A", "gdje"): 1,
     ("gdje", "je"): 2,
     ("je", "ured"): 1,
     ("ured", "</sent>"): 1,
     ("<sent>", "Ne"): 1,
     ("Ne", "znam"): 1,
     ("znam", "gdje"): 1,
     ("je", "</sent>": 1)
     }
    
brojač unigrama je

    {('<sent>',): 2, 
     ('A',): 1, 
     ('gdje',): 2, 
     ('je',): 2, 
     ('tvoj',): 1, 
     ('ured',): 1, 
     ('</sent>',): 2, 
     ('Ne',): 1, 
     ('znam',): 1}
     
model jezika je

    {('<sent>', 'A'): 0.5, 
     ('A', 'gdje'): 1.0, 
     ('gdje', 'je'): 1.0, 
     ('je', 'tvoj'): 0.5, 
     ('tvoj', 'ured'): 1.0,
     ('ured', '</sent>'): 1.0, 
     ('<sent>', 'Ne'): 0.5, 
     ('Ne', 'znam'): 1.0, 
     ('znam', 'gdje'): 1.0, 
     ('je', '</sent>'): 0.5})


In [8]:
def build_lang_model(txt, ngram_size):
    counter_hi = count_ngrams(txt, ngram_size=ngram_size)
    counter_lo = count_ngrams(txt, ngram_size=ngram_size - 1)
    
    model = {}
    for ngram in counter_hi:
        model[ngram] = counter_hi[ngram] / counter_lo[ngram[:-1]]
    return model

test(build_lang_model, ("A gdje je tvoj ured? Ne znam gdje je.", 1), {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385, ('je',): 0.15384615384615385, ('tvoj',): 0.07692307692307693, ('ured',): 0.07692307692307693, ('</sent>',): 0.15384615384615385, ('Ne',): 0.07692307692307693, ('znam',): 0.07692307692307693})
test(build_lang_model, ("A gdje je tvoj ured? Ne znam gdje je.", 2), {('<sent>', 'A'): 0.5, ('A', 'gdje'): 1.0, ('gdje', 'je'): 1.0, ('je', 'tvoj'): 0.5, ('tvoj', 'ured'): 1.0, ('ured', '</sent>'): 1.0, ('<sent>', 'Ne'): 0.5, ('Ne', 'znam'): 1.0, ('znam', 'gdje'): 1.0, ('je', '</sent>'): 0.5})
test(build_lang_model, ("A gdje je tvoj ured? Ne znam gdje je.", 3), {('<sent>', 'A', 'gdje'): 1.0, ('A', 'gdje', 'je'): 1.0, ('gdje', 'je', 'tvoj'): 0.5, ('je', 'tvoj', 'ured'): 1.0, ('tvoj', 'ured', '</sent>'): 1.0, ('<sent>', 'Ne', 'znam'): 1.0, ('Ne', 'znam', 'gdje'): 1.0, ('znam', 'gdje', 'je'): 1.0, ('gdje', 'je', '</sent>'): 0.5})

model1 = test_model(build_lang_model, (read_file(FILENAME), 1), {'2.12e-04': 1180, '4.24e-04': 183, '6.36e-04': 73, '8.47e-04': 44, '1.06e-03': 20, '1.27e-03': 8, '1.48e-03': 15, '1.69e-03': 13, '1.91e-03': 4, '2.12e-03': 3, '2.33e-03': 2, '2.54e-03': 3, '2.75e-03': 1, '2.97e-03': 3, '3.18e-03': 4, '3.39e-03': 2, '3.60e-03': 1, '3.81e-03': 1, '4.03e-03': 4, '4.24e-03': 1, '4.66e-03': 1, '5.72e-03': 1, '6.57e-03': 1, '7.20e-03': 1, '7.42e-03': 1, '8.47e-03': 1, '9.75e-03': 1, '9.96e-03': 1, '1.17e-02': 1, '2.06e-02': 1, '2.14e-02': 1, '2.31e-02': 1, '1.46e-01': 2})
model2 = test_model(build_lang_model, (read_file(FILENAME), 2), {'1.45e-03': 269, '2.91e-03': 47, '4.36e-03': 20, '5.81e-03': 13, '7.27e-03': 6, '8.72e-03': 4, '9.17e-03': 80, '9.90e-03': 49, '1.02e-02': 4, '1.03e-02': 48, '1.16e-02': 4, '1.31e-02': 1, '1.45e-02': 1, '1.82e-02': 29, '1.83e-02': 6, '1.98e-02': 7, '2.03e-02': 2, '2.06e-02': 6, '2.13e-02': 33, '2.17e-02': 35, '2.18e-02': 2, '2.50e-02': 36, '2.86e-02': 26, '2.94e-02': 23, '2.97e-02': 1, '3.09e-02': 2, '3.20e-02': 1, '3.23e-02': 26, '3.64e-02': 6, '3.67e-02': 2, '3.70e-02': 23, '3.96e-02': 2, '4.12e-02': 1, '4.26e-02': 7, '4.35e-02': 4, '4.55e-02': 14, '5.00e-02': 5, '5.26e-02': 44, '5.45e-02': 3, '5.56e-02': 16, '5.71e-02': 2, '5.88e-02': 11, '6.25e-02': 17, '6.45e-02': 1, '6.52e-02': 1, '6.67e-02': 39, '7.14e-02': 31, '7.22e-02': 1, '7.41e-02': 2, '7.69e-02': 13, '7.92e-02': 1, '8.26e-02': 1, '8.33e-02': 28, '8.91e-02': 1, '9.09e-02': 17, '9.68e-02': 1, '9.90e-02': 1, '1.00e-01': 12, '1.05e-01': 8, '1.11e-01': 33, '1.18e-01': 1, '1.25e-01': 67, '1.33e-01': 2, '1.43e-01': 81, '1.67e-01': 34, '1.76e-01': 2, '2.00e-01': 71, '2.06e-01': 1, '2.11e-01': 1, '2.22e-01': 2, '2.50e-01': 131, '2.63e-01': 1, '2.67e-01': 1, '2.73e-01': 2, '2.86e-01': 5, '3.33e-01': 142, '3.50e-01': 1, '3.64e-01': 1, '3.68e-01': 1, '3.75e-01': 2, '4.00e-01': 9, '4.29e-01': 3, '5.00e-01': 295, '5.71e-01': 2, '6.00e-01': 3, '6.25e-01': 1, '6.67e-01': 23, '7.50e-01': 8, '8.00e-01': 3, '9.38e-01': 1, '1.00e+00': 1239})
model3 = test_model(build_lang_model, (read_file(FILENAME), 3), {'4.55e-02': 14, '5.00e-02': 10, '6.67e-02': 25, '7.14e-02': 19, '9.09e-02': 1, '1.00e-01': 14, '1.11e-01': 19, '1.25e-01': 29, '1.33e-01': 3, '1.43e-01': 32, '1.67e-01': 18, '2.00e-01': 31, '2.22e-01': 4, '2.50e-01': 71, '2.67e-01': 1, '2.73e-01': 1, '2.86e-01': 2, '3.00e-01': 1, '3.33e-01': 75, '3.75e-01': 1, '4.00e-01': 3, '4.29e-01': 1, '4.67e-01': 1, '5.00e-01': 254, '5.71e-01': 1, '6.00e-01': 2, '6.67e-01': 9, '7.50e-01': 5, '8.00e-01': 3, '1.00e+00': 2516})

OK	build_lang_model('A gdje je tvoj ured? Ne znam gdje je.', 1) 
=> {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385}
== {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385, ('je',): 0.15384615384615385, ('tvoj',): 0.07692307692307693, ('ured',): 0.07692307692307693, ('</sent>',): 0.15384615384615385, ('Ne',): 0.07692307692307693, ('znam',): 0.07692307692307693}

OK	build_lang_model('A gdje je tvoj ured? Ne znam gdje je.', 2) 
=> {('<sent>', 'A'): 0.5, ('A', 'gdje'): 1.0, ('gdje', 'je'): 1.0}
== {('<sent>', 'A'): 0.5, ('A', 'gdje'): 1.0, ('gdje', 'je'): 1.0, ('je', 'tvoj'): 0.5, ('tvoj', 'ured'): 1.0, ('ured', '</sent>'): 1.0, ('<sent>', 'Ne'): 0.5, ('Ne', 'znam'): 1.0, ('znam', 'gdje'): 1.0, ('je', '</sent>'): 0.5}

OK	build_lang_model('A gdje je tvoj ured? Ne znam gdje je.', 3) 
=> {('<sent>', 'A', 'gdje'): 1.0, ('A', 'gdje', 'je'): 1.0, ('gdje', 'je', 'tvoj'): 0.5}
== {('<sent>', 'A', 'gdje'): 1

## 4.6 Vjerojarnost rečenice

Napravi funkciju **sent_prob()** po sljedećim ulaznim i izlaznim podacima
* ulaz
    * **txt** tekst rečenice
    * **model** model jezika kao rječnik čiji ključevi su ngrami, a vrijednosti njihove vjerojatnosti
* izlaz
    * vjeroratnost rečenice po modelu jezika 
    
Napomena: koristiti **build_sent_ngram()** za napraviti ngram rečenice. Za ngram_size je dovoljno uzeti bilo koji ngram iz modela i pogledati njegovu veličinu.
    

In [9]:
def sent_prob(sent, model):
    prob = 1
    ngram_size = len(next(iter(model)))
    for ngram in build_sent_ngram(sent, ngram_size=ngram_size):
        prob *= model[ngram]
    return prob
        

    
test(sent_prob, ("Hm... uvijek kad zakočim ispadne kotač i ne čeka.", model1), 8.755353678606873e-29)
test(sent_prob, ("U ovih nekoliko posljednjih sati, život se budio prolaznici su zaista dosadni!", model2), 1.9510958603052824e-09)
test(sent_prob, ("A gdje je tvoj ured?", model3), 0.022727272727272728)

OK	sent_prob('Hm... uvijek kad zakočim ispadne kotač i ne čeka.', {('<sent>',): 0.14576271186440679, ('Alan',): 0.003389830508474576, ('Ford',): 0.004025423728813559}) 
=> 8.755353678606873e-29
== 8.755353678606873e-29

OK	sent_prob('U ovih nekoliko posljednjih sati, život se budio prolaznici su zaista dosadni!', {('<sent>', 'Alan'): 0.007267441860465116, ('Alan', 'Ford'): 0.9375, ('Ford', '001'): 0.05263157894736842}) 
=> 1.9510958603052824e-09
== 1.9510958603052824e-09

OK	sent_prob('A gdje je tvoj ured?', {('<sent>', 'Alan', 'Ford'): 0.8, ('Alan', 'Ford', '001'): 0.06666666666666667, ('Ford', '001', '-'): 1.0}) 
=> 0.022727272727272728
== 0.022727272727272728



## 4.7. Izgradnja modela jezika po Laplace-u (dodaj-1 izglađivanje)

Napraviti klasu **LangModelLaplace** koja će imati sljedeće specijalne metode:

* `__init__(self, txt, ngram_size)` metoda koja će za dani **txt** i **ngram_size** stvoriti atribute:
    * `self.counter_hi` - brojač ngrama u tekstu `txt` za `ngram_size`. Koristiti **build_counter()**.
    * `self.counter_lo` - brojač ngrama u tekstu `txt` za `ngram_size - 1`. Koristiti **build_counter()**.
    * `v` - veličina rječnika za `txt` uvečana za 2 jer dodajemo `<sent>` i `</sent>`.  Koristiti **build_vocab()**
* `__getitem__(self, ngram)` metoda koja će za dani **ngram** vratiti vjerojatnost ngrama po Laplace-u. `self.counter_hi`sadrži brojač ngrama za brojnik, a `self.counter_lo`sadrži brojač ngrama za nazivnik. Ova specijalna metoda omogućava da se na instanci klase pristupa nekoj vrijednosti kao kod rječnika. Odnosno, ako je 
`model_laplace` instanca klase `LangModelLaplace`, onda se vrijednosti ngrama `ngram` pristupa s `model_laplace[ngram]`.

Napomena: za bigrame, vjerojatnost pojedinog bigrama $(w_{i-1}, w_i)$ je jednaka 

$$P(w_{i-1}, w_i) = \frac{broj((w_{i-1}, w_i)) + 1}{broj(w_{i-1}) + |V|}$$

gdje je $|V|$ veličina rječnika.
    
Napomena: za trigrame, vjerojatnost pojedinih trigrama $(w_{i-2}, w_{i-1}, w_i)$ je jednaka 

$$P(w_{i-2}, w_{i-1}, w_i) = \frac{broj((w_{i-2}, w_{i-1}, w_i) + 1}{broj(w_{i-2}, w_{i-1}) + |V|}$$

gdje je $|V|$ veličina rječnika.

Napravljena funkcija **build_lang_model_laplace()** jednostavno vraća instancu **LangModelLaplace** klase.

In [10]:
class LangModelLaplace(dict):
    
    def __init__(self, txt, ngram_size):
        self.counter_hi = count_ngrams(txt, ngram_size=ngram_size)
        self.counter_lo = count_ngrams(txt, ngram_size=ngram_size - 1)
        self.v = len(build_vocab(txt)) + 2
        for ngram in self.counter_hi:
            self[ngram] = self[ngram]
        
    def __getitem__(self, ngram):
        return (self.counter_hi.get(ngram, 0) + 1) / (self.counter_lo.get(ngram[:-1], 0) + self.v)


def build_lang_model_laplace(txt, ngram_size):
    return LangModelLaplace(txt, ngram_size)


testname(build_lang_model_laplace)

test(build_lang_model, ("A gdje je tvoj ured? Ne znam gdje je.", 1), {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385, ('je',): 0.15384615384615385, ('tvoj',): 0.07692307692307693, ('ured',): 0.07692307692307693, ('</sent>',): 0.15384615384615385, ('Ne',): 0.07692307692307693, ('znam',): 0.07692307692307693})
test(build_lang_model_laplace, ("A gdje je tvoj ured? Ne znam gdje je.", 1), {('<sent>',): 0.13636363636363635, ('A',): 0.09090909090909091, ('gdje',): 0.13636363636363635, ('je',): 0.13636363636363635, ('tvoj',): 0.09090909090909091, ('ured',): 0.09090909090909091, ('</sent>',): 0.13636363636363635, ('Ne',): 0.09090909090909091, ('znam',): 0.09090909090909091})

test(build_lang_model, ("A gdje je ured tvoj? Ne znam gdje je.", 2), {('<sent>', 'A'): 0.5, ('A', 'gdje'): 1.0, ('gdje', 'je'): 1.0, ('je', 'ured'): 0.5, ('ured', 'tvoj'): 1.0, ('tvoj', '</sent>'): 1.0, ('<sent>', 'Ne'): 0.5, ('Ne', 'znam'): 1.0, ('znam', 'gdje'): 1.0, ('je', '</sent>'): 0.5})
test(build_lang_model_laplace, ("A gdje je ured tvoj? Ne znam gdje je.", 2), {('<sent>', 'A'): 0.18181818181818182, ('A', 'gdje'): 0.2, ('gdje', 'je'): 0.2727272727272727, ('je', 'ured'): 0.18181818181818182, ('ured', 'tvoj'): 0.2, ('tvoj', '</sent>'): 0.2, ('<sent>', 'Ne'): 0.18181818181818182, ('Ne', 'znam'): 0.2, ('znam', 'gdje'): 0.2, ('je', '</sent>'): 0.18181818181818182})

model_laplace1 = test_model(build_lang_model_laplace, (read_file(FILENAME), 1), {'3.18e-04': 1180, '4.76e-04': 183, '6.35e-04': 73, '7.94e-04': 44, '9.53e-04': 20, '1.11e-03': 8, '1.27e-03': 15, '1.43e-03': 13, '1.59e-03': 4, '1.75e-03': 3, '1.91e-03': 2, '2.06e-03': 3, '2.22e-03': 1, '2.38e-03': 3, '2.54e-03': 4, '2.70e-03': 2, '2.86e-03': 1, '3.02e-03': 1, '3.18e-03': 4, '3.33e-03': 1, '3.65e-03': 1, '4.45e-03': 1, '5.08e-03': 1, '5.56e-03': 1, '5.72e-03': 1, '6.51e-03': 1, '7.46e-03': 1, '7.62e-03': 1, '8.89e-03': 1, '1.56e-02': 1, '1.62e-02': 1, '1.75e-02': 1, '1.09e-01': 2})
model_laplace2 = test_model(build_lang_model_laplace, (read_file(FILENAME), 2), {'8.82e-04': 269, '1.18e-03': 80, '1.19e-03': 48, '1.22e-03': 29, '1.23e-03': 35, '1.24e-03': 26, '1.25e-03': 39, '1.26e-03': 136, '1.27e-03': 1180, '1.32e-03': 47, '1.76e-03': 20, '1.78e-03': 6, '1.79e-03': 6, '1.84e-03': 6, '1.85e-03': 2, '1.86e-03': 1, '1.87e-03': 1, '1.88e-03': 2, '1.89e-03': 9, '1.90e-03': 44, '2.21e-03': 13, '2.38e-03': 1, '2.39e-03': 2, '2.45e-03': 3, '2.46e-03': 1, '2.48e-03': 1, '2.51e-03': 1, '2.52e-03': 3, '2.53e-03': 13, '2.65e-03': 6, '2.96e-03': 2, '2.98e-03': 1, '3.09e-03': 4, '3.13e-03': 1, '3.14e-03': 1, '3.15e-03': 2, '3.16e-03': 1, '3.53e-03': 4, '3.67e-03': 1, '3.72e-03': 1, '3.75e-03': 1, '3.78e-03': 1, '3.97e-03': 4, '4.37e-03': 1, '4.41e-03': 1, '4.42e-03': 1, '4.77e-03': 1, '4.85e-03': 1, '4.96e-03': 1, '5.00e-03': 1, '5.01e-03': 1, '5.02e-03': 1, '5.36e-03': 1, '5.66e-03': 1, '5.92e-03': 1, '5.95e-03': 1, '6.55e-03': 1, '6.62e-03': 2, '6.88e-03': 1, '6.90e-03': 1, '7.06e-03': 2, '1.00e-02': 1, '1.01e-02': 1, '1.25e-02': 1})
model_laplace3 = test_model(build_lang_model_laplace, (read_file(FILENAME), 3), {'1.25e-03': 25, '1.26e-03': 72, '1.27e-03': 2461, '1.87e-03': 1, '1.88e-03': 1, '1.89e-03': 2, '1.90e-03': 44, '2.51e-03': 1, '2.52e-03': 1, '2.53e-03': 8, '3.13e-03': 1, '3.14e-03': 1, '3.15e-03': 1, '3.16e-03': 2, '4.37e-03': 1, '4.42e-03': 2, '5.02e-03': 1, '5.66e-03': 1, '5.67e-03': 1})





BUILD_LANG_MODEL_LAPLACE
------------------------------------------------------------
OK	build_lang_model('A gdje je tvoj ured? Ne znam gdje je.', 1) 
=> {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385}
== {('<sent>',): 0.15384615384615385, ('A',): 0.07692307692307693, ('gdje',): 0.15384615384615385, ('je',): 0.15384615384615385, ('tvoj',): 0.07692307692307693, ('ured',): 0.07692307692307693, ('</sent>',): 0.15384615384615385, ('Ne',): 0.07692307692307693, ('znam',): 0.07692307692307693}

OK	build_lang_model_laplace('A gdje je tvoj ured? Ne znam gdje je.', 1) 
=> {('<sent>',): 0.13636363636363635, ('A',): 0.09090909090909091, ('gdje',): 0.13636363636363635}
== {('<sent>',): 0.13636363636363635, ('A',): 0.09090909090909091, ('gdje',): 0.13636363636363635, ('je',): 0.13636363636363635, ('tvoj',): 0.09090909090909091, ('ured',): 0.09090909090909091, ('</sent>',): 0.13636363636363635, ('Ne',): 0.09090909090909091, ('znam',): 0.090909090909090

## 4.8 Usporedba modela 

In [11]:
def print_comparison(txt):
    print(f"Rečenica: {txt}\n")
    print("          model\t\tmodel_laplace")
    for i, (model, model_laplace) in enumerate(zip([model1, model2, model3], [model_laplace1, model_laplace2, model_laplace3]), 1):

        try:
            prob = sent_prob(txt, model)
        except:
            prob = None
        prob_laplace = sent_prob(txt, model_laplace)

        if prob:
            print(f"{i}-gram:   {prob:.2e}\t{prob_laplace:.2e} ")
        else:
            print(f"{i}-gram:   \t\t{prob_laplace:.2e} ")
    print("-" * 40)

print_comparison("A gdje je tvoj ured?")
print_comparison("A gdje je ured tvoj?")
print_comparison("A gdje je banana?")

Rečenica: A gdje je tvoj ured?

          model		model_laplace
1-gram:   1.74e-16	8.24e-17 
2-gram:   6.67e-06	6.84e-17 
3-gram:   2.27e-02	3.21e-15 
----------------------------------------
Rečenica: A gdje je ured tvoj?

          model		model_laplace
1-gram:   1.74e-16	8.24e-17 
2-gram:   		5.70e-18 
3-gram:   		4.01e-16 
----------------------------------------
Rečenica: A gdje je banana?

          model		model_laplace
1-gram:   		5.77e-14 
2-gram:   		9.02e-15 
3-gram:   		6.33e-13 
----------------------------------------


## 4.9 Generiranje slučajnih rečenica

In [12]:
import random


def random_chain(model):
    word = '<sent>'
    sentence = []
    while word != '</sent>':
        next_grams = list({gram for gram in model if gram[0] == word})
        
        if next_grams:
            next_gram = random.choice(next_grams)
            sentence.append(next_gram)
            word = next_gram[-1]
            if word == '<sent>':
                break
        else:
            break
    # print(sentence)
    sentence = ' '.join(' '.join(gram[1:]) for gram in sentence).strip('</sent>')
    print(sentence)

for _ in range(20):
    random_chain(model3)

Alane Alane ja putujem u New York 
Upravo tako miroljubljivi 
Ne pušim
Hm u New Yorku
U devet sati 
Nancy je pokojnik bio policajac i dva puta da se zaista zove Alan Ford Porb ili jednih i smaragd 
Vratio sam glupa i bajoslovno vrijedni smaragd 
Odviše si zaposlena čitavu noć 
Zapamti ubijati treba golim rukama 
I možemo odmah dalje situacija je čula moj glas spustila slušalicu 
Poslije gimnastike na svježem zraku neće iznevjeriti 
Ovi su me zamijenili s nekim zamijenili 
Tebi ne brbljaj 
Ili platite ili u samostan nego što dobije brzinu 
Zaista ste Margot 
Nemam ništa jesti 
Čuo sam za rek
Ah sat je još toga 
Ta moderna muzika zaista ono za reklamu i po 
I možemo odmah dalje situacija je težak prekršaj kopčaš 


## 4.10. Perpleksija

Inkluzivna evaluacija modela jezika se provodi perpleksijom čija je formula:

$$PP(w_1, ..., w_N) = \sqrt[N]{\frac{1}{p(w_1)...p(w_N)}}=\sqrt[N]{\frac{1}{\sum^N_1{p(w_i)}}}$$

Zbog prelijeva decimalne točke, perpleksija se praktički računa u logaritamskom prostoru.

$$log(PP(w_1, ..., w_N))=\frac{-1}{N}\sum^N_1{log(p(w_i))}$$


In [13]:
import math

def perplexity(txt, model):
    ngram_size = len(next(iter(model)))

    vocab_size = len(build_vocab(txt))    
    p = 0
    for sent in segment_sent(txt):
        for ngram in build_sent_ngram(sent, ngram_size):
            pi = model[ngram]
            p += math.log(pi)    

    pp = (-1 / vocab_size) * p
    return math.exp(pp)
        
test_txt = """
Bio bih sretan 
Kakve su policajci 
Jedino nedostaje mikrofilm a sad evo me čuj momče 
Ali to je mnogo struju 
Hvala mladiću 
Prava pravcata mušterija advokata i tu piše 
Mister Ford 
Zovem je sinko 
Prljavi izdajniče 
Nešto u posljednji čas Miss 
Palm Beach je čim je grad pun nevaljalaca 
Želiš li razočaranja 
New York 
Stare su nas 
Razumije se rezervacija za kandidata smrti 
Znam malajski trik za početak poslije ću ti ja se prljavi štakore 
Kod nje je naša blagajna punija 
Moramo hitno u dnevnom tisku za početak poslije ću zadržati za petama žutokljunče 
Glupane 
Oprosti na stvari 
"""
    
train_txt = read_file(FILENAME)

for ngram_size in range(1, 5):
    model = build_lang_model_laplace(train_txt, ngram_size)
    pp = perplexity(test_txt,  model)
    print(f"{ngram_size}-gram\tPP = {pp}")

1-gram	PP = 9528.02408134646
2-gram	PP = 9997.350460129475
3-gram	PP = 2878.8639045661016
4-gram	PP = 731.6622279029574
